# OpenGridMap Notebook

# Step1. Preparation

## Import necessary modules

In [164]:
import psycopg2
import json
import ipyleaflet as L
import networkx as nx
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl
)

## Database connection

In [199]:
conn.close()
# Set connection to gis db
try:
    conn = psycopg2.connect("dbname=gis user=jennyzhou")
except:
    print 'Fail to connect to Postgres Server'

# Step2. Data selection

## Select area of Garching on map

In [166]:
# Select polygon of Garching
cur = conn.cursor() 
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))), ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM planet_osm_polygon WHERE osm_id = -30971; ")
garching = cur.fetchall()
cur.close()

garchingCentJson = json.loads(garching[0][0])
garchingPolJson = json.loads(garching[0][1])
#coor = json.loads(garching[0][1])['coordinates'][0]

## Select transformer entries

In [167]:
cur = conn.cursor()
# Select all the transformers inside Garching
cur.execute("SELECT ST_AsGeoJSON(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326)))) FROM student_transformers;")
transformers = cur.fetchall()
cur.close()
transJson = json.loads(transformers[0][0])

#cur.execute("CREATE TABLE transformer_geo (id serial PRIMARY KEY, osm_id bigint, geo_json text);")
#cur.execute("INSERT INTO transformer_geo (osm_id, geo_json) SELECT osm_id, ST_AsGeoJSON(ST_Transform(way, 4326)) FROM planet_osm_point WHERE power != ''ORDER BY name;")

## Voronoi segmentation within Garching

In [212]:
cur = conn.cursor()
# Get voronoi diagram
cur.execute("SELECT ST_AsGeoJSON(ST_VoronoiPolygons(ST_Collect(ST_FlipCoordinates(ST_Transform(way, 4326))))) FROM student_transformers;")
voronoi = cur.fetchall()
voronoiJson = json.loads(voronoi[0][0])

## Select house entries

In [169]:
cur = conn.cursor()
# Too Slow
cur.execute("SELECT ST_AsGeoJson(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326))) FROM planet_osm_polygon WHERE ST_Within(way,(SELECT way FROM planet_osm_polygon WHERE osm_id = -30971)) AND building != '' LIMIT 10; ")
houses = cur.fetchall()
cur.close()

# Step3. Data visualisation with ipyleaflet

## 3.1 General display with all data

In [213]:
# Initial map container with Garching as center
map = Map(default_tiles=TileLayer(opacity=1.0),center= garchingCentJson['coordinates'],zoom =13)
map

### Mark boundary of Garching

In [214]:
# Show boundary of Garching
garchingOuter = Polygon(locations=garchingPolJson['coordinates'], weight=4,
            color='#003d99', opacity=0.8, fill_opacity=0.0,
            fill_color='#ccffcc')
map += garchingOuter

### Voronoi Diagram

In [215]:
for poly in voronoiJson['geometries']:
    pg = Polygon(locations=poly['coordinates'], weight=2,
            color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
            fill_color='#ccffcc') 
    map.add_layer(pg)

### Mark transformers

In [216]:
# Mark transformers
tmarkers = []
for tp in transJson['coordinates']:  
    mk = Marker(location=tp)
    tmarkers.append(mk)
    map.add_layer(mk)

In [175]:
# Clean Markers
for mk in tmarkers:
    map.remove_layer(mk)

## 3.2 Testing within one voronoi polygon

In [217]:
# Fix area -> good choice: 38 (or 37) 
tArea = voronoiJson['geometries'][38]

In [237]:
# Find corresponding transformers
tGeoJson = json.dumps(tArea)
cur = conn.cursor()
cur.execute("SELECT ST_AsGeoJSON(ST_FlipCoordinates(ST_Transform(way, 4326))) FROM student_transformers WHERE ST_Within(ST_FlipCoordinates(ST_Transform(way, 4326)),ST_SetSRID(ST_GeomFromGeoJSON(\'%s\'),4326));" % tGeoJson)
tTrans = cur.fetchall()

# Find corresponding houses
cur.execute("SELECT ST_AsGeoJson(ST_Collect(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326)))) FROM planet_osm_polygon WHERE ST_Within(ST_FlipCoordinates(ST_Transform(ST_Centroid(way), 4326)),ST_SetSRID(ST_GeomFromGeoJSON(\'%s\'),4326)) AND building != '';" % tGeoJson)
houses = cur.fetchall()
cur.close()
tTransJson = json.loads(tTrans[0][0])
tHouseJson = json.loads(houses[0][0])


In [176]:
# Mark houses
hmarkers = []
for row in houses:
    c = Circle(location=json.loads(row[0])['coordinates'], weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    map.add_layer(c) 

In [222]:
# Mark transformer
tCenter = tTransJson['coordinates']
tMarker =  Marker(location=tCenter)

# Mark polygon
tPoly = Polygon(locations=tArea['coordinates'], weight=2,
            color='#a3c2c2', opacity=0.8, fill_opacity=0.2,
            fill_color='#ccffcc') 

# Init Map
tMap = Map(default_tiles=TileLayer(opacity=1.0),center=tCenter ,zoom=16)
tMap.add_layer(tMarker)
tMap.add_layer(tPoly)
tMap

In [246]:
# Mark houses
hmarkers = []
for hp in tHouseJson['coordinates']:
    c = Circle(location= hp, weight=5, opacity = 0.5, color = 'red', radius = 2) 
    hmarkers.append(c)
    tMap.add_layer(c) 

In [ ]:
for mk in hmarkers:
    tMap.remove_layer(mk)

# Step4. Find best graph

## Preparing vertices and edges

In [ ]:
''' 
Transformer : tTransJson['coordinates']
Houses : tHouseJson['coordinates']
'''


## Build graph with data set

In [4]:
# To be implemented
def buildGraph(graph):
    # graph ["1 2 10.1", "2 3 5.3"]
    G=nx.parse_edgelist(graph, nodetype = int, data=(('weight',float),))
    return G


In [5]:
# Testing Code
graph = ["1 2 10.1", "2 3 5.3"]
G = buildGraph(graph)
print G.nodes()
print G.edges(data = True)

[1, 2, 3]
[(1, 2, {'weight': 10.1}), (2, 3, {'weight': 5.3})]


## Run network algorithm

In [ ]:
# To be implemented

# Step5. Display graph on map